In [15]:
from datetime import datetime
from io import BytesIO
from typing import List
import os
import pymongo
from bson import ObjectId
from deepface import DeepFace
from dotenv import load_dotenv
from fastapi import APIRouter
from fastapi import File
from fastapi import HTTPException
from fastapi import Response
from fastapi import UploadFile
from matplotlib import pyplot as plt
from PIL import Image
from pydantic import BaseModel

from API.database import Database
from API.utils import init_logging_config

load_dotenv(dotenv_path="/API/.env")

False

In [27]:
MONGO_URI = os.getenv('MONGO_URL1')
router = APIRouter()


client = Database()
client2 = Database(MONGO_URI, 'FaceRec')

collection = 'faceEntries'
collection1 = 'ImageDB'
collection2 = 'VectorDB'


In [30]:
employees_mongo = client.find(collection2)
for employee in employees_mongo:
    print(employee)

In [39]:
def insert_data(name, image_list, client, collection):
    # fetch all the entries from the collection
    entries = client.find(collection)
    if entries is None:
        entries = []
    # check if the name is already in the collection
    names = [entry['name'] for entry in entries]
    if name in names:
        for image in image_list:
            embedding = DeepFace.represent(image, model_name='Facenet512', detector_backend='mtcnn')
            print(embedding[0]["embedding"])
            if float(embedding[0]["face_confidence"])>0.8:
                print("Inserting in DB")
                client.update_one(collection, {'name': name}, {'$push': {'embeddings': embedding}})
    else:
        # create a new entry
        embeddings = []
        for image in image_list:
            embedding = DeepFace.represent(image, model_name='Facenet512', detector_backend='mtcnn')
            if float(embedding[0]["face_confidence"])>0.8:
                print("Inserting in DB")
                embeddings.append(embedding)
        client.insert_one(collection, {'name': name, 'embeddings': embeddings})

In [40]:
name = 'Devansh Shah'
image_list = ['Images/devansh.jpg', 'Images/dbImages/Timepass.jpg']

insert_data(name, image_list, client, collection2)

1/1 [==============================] - 0s 31ms/step
[-0.8594350218772888, 1.631603717803955, -0.2927672863006592, -0.6287406086921692, -1.063706398010254, -0.4130912721157074, 0.002039658837020397, 2.285496234893799, 1.6468628644943237, -1.061055302619934, 0.04052991792559624, -0.4282776117324829, -2.2576937675476074, 0.0903540700674057, 0.31466907262802124, -0.3557279109954834, 1.0764038562774658, 1.0275729894638062, -0.7433256506919861, -1.364379644393921, -0.06946221739053726, 0.7525165677070618, 1.8126814365386963, -0.7196546792984009, -0.31789976358413696, -0.8317980170249939, -0.22326360642910004, 0.3996230661869049, 1.1184463500976562, 0.6761944890022278, -1.4526052474975586, 0.8621320128440857, 0.9493192434310913, 0.8537136316299438, 2.0761876106262207, 2.6479907035827637, -0.3922588527202606, 0.3036455512046814, 0.8079169988632202, 1.1987236738204956, 1.8728218078613281, -1.8342465162277222, -0.2588633894920349, -0.26962810754776, -0.5637434720993042, -1.4128532409667969, -1.5